# Selecting samples of 10k articles from CC_News with 5 diversity metrics:
1. [Setup / Removing non-English articles from the dataset](#setup)
2. [Random](#random)
3. [Mean TFIDF](#meantfidf)
4. [Jaccard similarity](#jaccard) -- ERROR: jaccard similarity not successfully scaled to CC_News dataset
5. [Trigram language model entropy](#3entropy)
6. [Measures of embedding diversity:](#embedding)
    1. [Outlier](#outlier)
    2. [Word Embedding Diversity](#wed)
<br>
*all metrics were computed, and models subsequently trained, using CPU on a macbook with M1 pro chip

In [1]:
__author__ = "Jon Ball"
__version__ = "CS224U, Stanford, Spring 2022"

Setup modules:

In [2]:
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import utils
import json
import time
import os

For reproducibility:

In [3]:
utils.fix_random_seeds()

Data:

In [4]:
ccn = load_dataset("cc_news", split="train")["text"]

Reusing dataset cc_news (/Users/jball/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6)


In [5]:
print(f"Number of articles: {len(ccn)}\n")
print(ccn[0])

Number of articles: 708241

There's a surprising twist to Regina Willoughby's last season with Columbia City Ballet: It's also her 18-year-old daughter Melina's first season with the company. Regina, 40, will retire from the stage in March, just as her daughter starts her own career as a trainee. But for this one season, they're sharing the stage together.
Performing Side-By-Side In The Nutcracker
Regina and Melina are not only dancing in the same Nutcracker this month, they're onstage at the same time: Regina is doing Snow Queen, while Melina is in the snow corps, and they're both in the Arabian divertissement. "It's very surreal to be dancing it together," says Regina. "I don't know that I ever thought Melina would take ballet this far."
Left: Regina and Melina with another company member post-snow scene in 2003. Right: The pair post-snow scene in 2017 (in the same theater)
Keep reading at dancemagazine.com.


### Check that the CC_News articles are actually in English using <a href="https://github.com/google/cld3">gcld3</a>: <a id="setup"></a>

In [6]:
import gcld3

In [7]:
classifier = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=100000)

In [8]:
for idx, article in enumerate(ccn):
    result = classifier.FindLanguage(text=article)
    if result.language != "en":
        print(idx, "\n", article)
        break

89 
 ০ = 0, ১ = 1, ২ = 2, ৩ = 3, ৪ = 4, ৫ = 5, ৬ = 6, ৭ = 7, ৮ = 8, ৯ = 9
্ = See example (Hasant/Viram) ় = * (Nukta) ʼ = ' (Urdhacomma) ঽ = & (Avagrah) ৺ = ~ (Isshar) ৹ = a~ (Bengali ana sign) ৲ = Rs~ (Bengali Rupee sign) ৳ = T~ (Taka sign) । = | (Devanagari danda) ॥ = || (Devanagari double danda) ₹ = Rs (Indian Rupee sign) 卐 = +~ (Swastika sign) Zero Width Joiner = ^ Zero Width Non Joiner = ^^
These symbols will type Bengali characters first but if "~" will be followed, it will remove previously typed Bengali character and then type the symbol.
Symbols & ~ * : ^ | ' have special meaning. You can type this way & = &~ ~ = ~~ * = *~ : = :~ ^ = ^~ | = |~ ' = '~
The English symbols [ ] { } ( ) < > - + / = ; . , " ? ! % \ _ $ @ # translate into the same symbols.
Example নমস্কার can be written by typing "namaskaar"
As per Rule # 3, ligature will be rendered. ZWJ and ZWNJ characters are used to produce alternate rendering of ligature.
A consonant followed by ZWJ character will produce half-

#### Use gcld3 to filter out articles like the above which are not fully English and likely to be included in each sample:

In [9]:
disinclusion_index = []
for idx, article in enumerate(tqdm(ccn)):
    result = classifier.FindLanguage(text=article)
    if result.language != "en":
        disinclusion_index.append(idx)

100%|█████████████████████████████████| 708241/708241 [06:19<00:00, 1867.32it/s]


In [10]:
%%time

cc_news = [
    article for idx, article in enumerate(ccn) if idx not in disinclusion_index
]

CPU times: user 21 s, sys: 33.1 ms, total: 21.1 s
Wall time: 21.1 s


In [11]:
print(f"Number of clean articles: {len(cc_news)}")

Number of clean articles: 703532


In [12]:
with open(os.path.join("data", "cc_news.json"), "w") as outfile:
    json.dump(cc_news, outfile)

## Mean-TFIDF (variation on Baeza-Yates et al. 1999) <a id="meanidf"></a>

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from operator import itemgetter
import numpy as np
import json
import time
import os

In [2]:
with open(os.path.join("data","cc_news.json"), "r") as infile:
    cc_news = json.load(infile)

print(len(cc_news))

703532


In [3]:
tfidf_vec = TfidfVectorizer()

In [4]:
%time tfidf_matrix = tfidf_vec.fit_transform(cc_news)

CPU times: user 1min 36s, sys: 2.57 s, total: 1min 39s
Wall time: 1min 39s


In [5]:
def mean_idf(doc_idx, tfidf_matrix=tfidf_matrix, tfidf_vectorizer=tfidf_vec):
    feature_index = tfidf_matrix[doc_idx,:].nonzero()[1]
    word_idf_scores = tfidf_vectorizer.idf_[feature_index]
    return np.mean(word_idf_scores)

In [6]:
%time cc_idf = [mean_idf(doc_idx) for doc_idx in range(len(cc_news))]

CPU times: user 19min 37s, sys: 5min 57s, total: 25min 34s
Wall time: 25min 35s


In [7]:
print(f"""Max IDF: {max(cc_idf)}
Min IDF: {min(cc_idf)}
Mean: {np.mean(cc_idf)}
Std Dev: {np.std(cc_idf)}""")

Max IDF: 12.365534427447304
Min IDF: 2.2256180368294363
Mean: 4.42180964113847
Std Dev: 0.5235009566338252


In [8]:
idf_indexed = list(enumerate(cc_idf))
idf_indexed10k = sorted(idf_indexed, key=itemgetter(1))[-10000:] #indices and top 10,000 Mean IDF scores
idf_indices10k = [tup[0] for tup in idf_indexed10k]
idf_scores10k = [tup[1] for tup in idf_indexed10k]

In [9]:
print(f"""Max IDF in subsample: {max(idf_scores10k)}
Min IDF in subsample: {min(idf_scores10k)}
Mean: {np.mean(idf_scores10k)}
Std Dev: {np.std(idf_scores10k)}""")

Max IDF in subsample: 12.365534427447304
Min IDF in subsample: 5.776707236449172
Mean: 6.319554792407199
Std Dev: 0.5852270273612017


In [10]:
%%time

cc_idf10k = [
    article for idx, article in enumerate(cc_news) if idx in idf_indices10k
]

CPU times: user 57 s, sys: 544 ms, total: 57.5 s
Wall time: 57.5 s


In [11]:
with open(os.path.join("data", "cc_idf10k.json"), "w") as outfile:
    json.dump(cc_idf10k, outfile)

## Jaccard distance <a id="jaccard"></a>

In [1]:
from tokenizers import BertWordPieceTokenizer
from operator import itemgetter
from tqdm import tqdm

import multiprocess as mp
import numpy as np
import time
import json
import os

In [2]:
with open(os.path.join("data","cc_news.json"), "r") as infile:
    cc_news = json.load(infile)

print(len(cc_news))

703532


In [3]:
tokenizer = BertWordPieceTokenizer(os.path.join("bert-base-uncased-vocab", "vocab.txt"))

In [4]:
cc_ids = tuple(
    [
        (article_idx, set(tokenizer.encode(cc_news[article_idx]).ids[:512]))\
        for article_idx in tqdm(range(len(cc_news)))
    ]
)

100%|█████████████████████████████████| 703532/703532 [09:11<00:00, 1275.09it/s]


In [5]:
del cc_news, tokenizer, BertWordPieceTokenizer

In [6]:
def mean_jaccard_dist(article_idx, token_set):
    
    jdistances = [
        1.0 - len(token_set.intersection(ref_tuple[1])) / len(token_set.union(ref_tuple[1])) \
        for ref_tuple in cc_ids if article_idx != ref_tuple[0]
    ]
    
    return (article_idx, np.mean(jdistances))

In [ ]:
Manually updated

In [7]:
%%time

if __name__ == "__main__":
    
    with mp.Pool(8) as pool:
        cc_jaccard = pool.starmap(mean_jaccard_dist, cc_ids)

KeyboardInterrupt: 

## Trigram language model entropy <a id="3entropy"></a>

In [1]:
from tokenizers import BertWordPieceTokenizer
from nltk.lm.preprocessing import pad_both_ends, flatten
from nltk.util import ngrams
from nltk.lm import MLE
from operator import itemgetter
from tqdm import tqdm
import numpy as np
import time
import json
import os

In [2]:
with open(os.path.join("data","cc_news.json"), "r") as infile:
    cc_news = json.load(infile)

print(len(cc_news))

703532


In [3]:
tokenizer = BertWordPieceTokenizer(os.path.join("bert-base-uncased-vocab", "vocab.txt"))

In [4]:
cc_toks = [
    tokenizer.encode(cc_news[article_idx]).tokens[:512] for article_idx in tqdm(range(len(cc_news)))
]

100%|█████████████████████████████████| 703532/703532 [10:15<00:00, 1143.80it/s]


In [5]:
%%time

cc_padded = [
    list(pad_both_ends(article_toks, n=3)) for article_toks in cc_toks
]

CPU times: user 10.3 s, sys: 57.3 s, total: 1min 7s
Wall time: 1min 45s


In [6]:
cc_vocab = list(flatten(cc_padded))
print(len(cc_vocab))

239677179


In [7]:
%%time

cc_grams = [
    list(ngrams(article_padded, n=3)) for article_padded in cc_padded
]

CPU times: user 26.6 s, sys: 2min 19s, total: 2min 46s
Wall time: 5min 59s


In [8]:
trigram_model = MLE(3)

In [9]:
%time trigram_model.fit(cc_grams, cc_vocab)

CPU times: user 16min 42s, sys: 19min, total: 35min 43s
Wall time: 1h 32s


In [10]:
cc_entropy = [
    trigram_model.entropy(cc_grams[article_idx]) for article_idx in tqdm(range(len(cc_grams)))
]

100%|██████████████████████████████████| 703532/703532 [56:06<00:00, 208.96it/s]


In [11]:
print(f"""Max entropy: {max(cc_entropy)}
Min entropy: {min(cc_entropy)}
Mean: {np.mean(cc_entropy)}
Std Dev: {np.std(cc_entropy)}""")

Max entropy: 6.441736870435621
Min entropy: 0.6607880940607269
Mean: 4.531580789229363
Std Dev: 0.9035417963006608


In [12]:
entropy_indexed = list(enumerate(cc_entropy))
entropy_indexed10k = sorted(entropy_indexed, key=itemgetter(1))[-10000:]
entropy_indices10k = [tup[0] for tup in entropy_indexed10k]
entropy_scores10k = [tup[1] for tup in entropy_indexed10k]

In [13]:
print(f"""Max entropy in subsample: {max(entropy_scores10k)}
Min entropy in subsample: {min(entropy_scores10k)}
Mean: {np.mean(entropy_scores10k)}
Std Dev: {np.std(entropy_scores10k)}""")

Max entropy in subsample: 6.441736870435621
Min entropy in subsample: 5.589578462962402
Mean: 5.680277063155953
Std Dev: 0.08761192165393417


In [14]:
%%time

cc_entropy10k = [
    article for idx, article in enumerate(cc_news) if idx in entropy_indices10k
]

CPU times: user 57.7 s, sys: 2.08 s, total: 59.8 s
Wall time: 1min 4s


In [15]:
with open(os.path.join("data", "cc_entropy10k.json"), "w") as outfile:
    json.dump(cc_entropy10k, outfile)

In [16]:
with open(os.path.join("data", "cc_entropy10k.txt"), "w") as outfile:
    for article in cc_entropy10k:
        outfile.write(article)

## Get embeddings <a id="embedding"></a>

In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
import json
import time
import os

In [2]:
with open(os.path.join("data","cc_news.json"), "r") as infile:
    cc_news = json.load(infile)

print(len(cc_news))

703532


In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
%%time
#using BertTokenizer rather than BertTokenizerFast to avoid forking parallelism issue with for-loop below
tokenized_inputs = [
    tokenizer(cc_news[doc_idx], truncation=True, max_length=512, padding="max_length", return_tensors="pt")\
    for doc_idx in range(len(cc_news))
]

CPU times: user 50min 53s, sys: 15 s, total: 51min 8s
Wall time: 51min 9s


In [5]:
del cc_news

First pass (interrupted due to power outage):

In [6]:
def get_mean_pooled_array(article_idx, tokenized_inputs=tokenized_inputs):
    
    input_ = tokenized_inputs[article_idx]
    
    output_ = model(**input_, output_hidden_states=True)
    
    out_tensor = torch.mean(output_.last_hidden_state, axis=1)
    
    return out_tensor.detach().numpy()

In [7]:
%%time

for article_idx in tqdm(range(len(tokenized_inputs))):
    
    arr = get_mean_pooled_array(article_idx)
    filename = "cc" + str(article_idx) + ".npy"
    np.save(os.path.join("data", "cc_arrays", filename), arr)

 50%|██████████████▌              | 352567/703532 [76:22:02<76:01:12,  1.28it/s]


KeyboardInterrupt: 

Second pass:

In [8]:
def get_mean_pooled_array(article_toks, model=model):
    
    output_ = model(**article_toks, output_hidden_states=True)
    
    out_tensor = torch.mean(output_.last_hidden_state, axis=1)
    
    return out_tensor.detach().numpy()

In [9]:
%%time

for new_idx, article_toks in tqdm(enumerate(tokenized_inputs[352567:])):
    
    arr = get_mean_pooled_array(article_toks)
    filename = "cc" + str(new_idx + 352567) + ".npy"
    np.save(os.path.join("data", "cc_arrays", filename), arr)

350965it [85:28:24,  1.14it/s]

CPU times: user 26d 4min 32s, sys: 4d 6h 18min 12s, total: 30d 6h 22min 44s
Wall time: 3d 13h 28min 24s


## Euclidean distance from mean corpus vector (Stasaski et al., 2020; Larson et al., 2019)

In [1]:
from scipy.spatial.distance import euclidean
from operator import itemgetter
from tqdm import tqdm
import numpy as np
import json
import time
import os

In [2]:
arr_path = os.path.join("data", "cc_arrays")

In [3]:
for roots, dirs, files in os.walk(arr_path):
    
    print(len(files))

    indices = [
        int(filename[2:-4]) for filename in files
    ]
    
    filepaths = [
        os.path.join(arr_path, filename) for filename in files
    ]
    
    idx2path = sorted(
        list(zip(indices, filepaths)),
        key = itemgetter(0)
    )
    
    print(idx2path[0])

703532
(0, 'data/cc_arrays/cc0.npy')


In [4]:
cc_array = np.zeros((len(idx2path), 768))

for idx, filepath in tqdm(idx2path):
    
    arr = np.load(filepath)
    cc_array[idx] = arr
        
print(cc_array.shape)

100%|█████████████████████████████████| 703532/703532 [01:44<00:00, 6734.86it/s]

(703532, 768)


In [5]:
mean_arr = np.mean(cc_array, axis=0)

In [6]:
cc_eudist = []

for idx in tqdm(range(cc_array.shape[0])):
    eudist = euclidean(cc_array[idx], mean_arr)
    cc_eudist.append(eudist)

100%|███████████████████████████████| 703532/703532 [00:02<00:00, 288114.68it/s]


In [7]:
print(f"""Max cosine distance from corpus mean: {max(cc_eudist)}
Min: {min(cc_eudist)}
Mean: {np.mean(cc_eudist)}
Std Dev: {np.std(cc_eudist)}""")

Max cosine distance from corpus mean: 13.397324553882964
Min: 1.960511518807967
Mean: 4.003239536662265
Std Dev: 0.8837525446080264


In [8]:
eudist_indexed = list(enumerate(cc_eudist))
eudist_indexed10k = sorted(eudist_indexed, key=itemgetter(1))[-10000:]
eudist_indices10k = [tup[0] for tup in eudist_indexed10k]
eudist_scores10k = [tup[1] for tup in eudist_indexed10k]

In [9]:
print(f"""Max euclidean distance in subsample: {max(eudist_scores10k)}
Min: {min(eudist_scores10k)}
Mean: {np.mean(eudist_scores10k)}
Std Dev: {np.std(eudist_scores10k)}""")

Max euclidean distance in subsample: 13.397324553882964
Min: 6.509962991455852
Mean: 6.921735291458649
Std Dev: 0.5148770026302417


In [10]:
with open(os.path.join("data","cc_news.json"), "r") as infile:
    cc_news = json.load(infile)

print(len(cc_news))

703532


In [11]:
%%time

cc_eudist10k = [
    article for idx, article in enumerate(cc_news) if idx in eudist_indices10k
]

CPU times: user 48.1 s, sys: 902 ms, total: 49 s
Wall time: 49 s


In [12]:
with open(os.path.join("data", "cc_eudist10k.json"), "w") as outfile:
    json.dump(cc_eudist10k, outfile)

In [13]:
with open(os.path.join("data", "cc_eudist10k.txt"), "w") as outfile:
    json.dump(cc_eudist10k, outfile)

## Word Embedding Diversity / Mean pairwise cosine distance (Palumbo et al. 2020)

In [1]:
from sklearn.metrics import pairwise_distances_chunked
from operator import itemgetter
from itertools import chain
from tqdm import tqdm
import numpy as np
import json
import time
import os

In [2]:
arr_path = os.path.join("data", "cc_arrays")

In [3]:
for roots, dirs, files in os.walk(arr_path):
    
    print(len(files))

    indices = [
        int(filename[2:-4]) for filename in files
    ]
    
    filepaths = [
        os.path.join(arr_path, filename) for filename in files
    ]
    
    idx2path = sorted(
        list(zip(indices, filepaths)),
        key = itemgetter(0)
    )
    
    print(idx2path[-1])

703532
(703531, 'data/cc_arrays/cc703531.npy')


In [4]:
cc_array = np.zeros((len(idx2path), 768))

for idx, filepath in tqdm(idx2path):
    
    arr = np.load(filepath)
    cc_array[idx] = arr

100%|█████████████████████████████████| 703532/703532 [01:41<00:00, 6928.59it/s]


In [5]:
%%time

cc_wed_chunked = []
it = 0
tock = time.time()
m_start = 0
m_end = 0

for chunk in pairwise_distances_chunked(cc_array, metric="cosine"):
    
    cc_wed_chunked.append(np.mean(chunk, axis=1).tolist())
    
    it += 1
    tick = time.time()
    m_start = m_end
    m_end += chunk.shape[0]
    
    print(f"Iter {it}  :  Rows {m_start}-{m_end}  :  {tick - tock} seconds elapsed")
    tock = time.time()

Iter 1  :  Rows 0-190  :  2.7233941555023193 seconds elapsed
Iter 2  :  Rows 190-380  :  3.185492992401123 seconds elapsed
Iter 3  :  Rows 380-570  :  2.486224889755249 seconds elapsed
Iter 4  :  Rows 570-760  :  2.200418710708618 seconds elapsed
Iter 5  :  Rows 760-950  :  2.2093570232391357 seconds elapsed
Iter 6  :  Rows 950-1140  :  2.2308738231658936 seconds elapsed
Iter 7  :  Rows 1140-1330  :  2.2043309211730957 seconds elapsed
Iter 8  :  Rows 1330-1520  :  2.2553868293762207 seconds elapsed
Iter 9  :  Rows 1520-1710  :  2.2104079723358154 seconds elapsed
Iter 10  :  Rows 1710-1900  :  2.191317081451416 seconds elapsed
Iter 11  :  Rows 1900-2090  :  2.2452123165130615 seconds elapsed
Iter 12  :  Rows 2090-2280  :  2.214040994644165 seconds elapsed
Iter 13  :  Rows 2280-2470  :  2.2247989177703857 seconds elapsed
Iter 14  :  Rows 2470-2660  :  2.2463490962982178 seconds elapsed
Iter 15  :  Rows 2660-2850  :  2.229434013366699 seconds elapsed
Iter 16  :  Rows 2850-3040  :  2.22113

Iter 124  :  Rows 23370-23560  :  2.2567250728607178 seconds elapsed
Iter 125  :  Rows 23560-23750  :  2.230362892150879 seconds elapsed
Iter 126  :  Rows 23750-23940  :  2.227579116821289 seconds elapsed
Iter 127  :  Rows 23940-24130  :  2.211789846420288 seconds elapsed
Iter 128  :  Rows 24130-24320  :  2.211935043334961 seconds elapsed
Iter 129  :  Rows 24320-24510  :  2.222219944000244 seconds elapsed
Iter 130  :  Rows 24510-24700  :  2.2193939685821533 seconds elapsed
Iter 131  :  Rows 24700-24890  :  2.190221071243286 seconds elapsed
Iter 132  :  Rows 24890-25080  :  2.2092318534851074 seconds elapsed
Iter 133  :  Rows 25080-25270  :  2.199831962585449 seconds elapsed
Iter 134  :  Rows 25270-25460  :  2.2603700160980225 seconds elapsed
Iter 135  :  Rows 25460-25650  :  2.2467548847198486 seconds elapsed
Iter 136  :  Rows 25650-25840  :  2.2310068607330322 seconds elapsed
Iter 137  :  Rows 25840-26030  :  2.2088911533355713 seconds elapsed
Iter 138  :  Rows 26030-26220  :  2.20877

Iter 244  :  Rows 46170-46360  :  2.2400572299957275 seconds elapsed
Iter 245  :  Rows 46360-46550  :  2.2135531902313232 seconds elapsed
Iter 246  :  Rows 46550-46740  :  2.218379020690918 seconds elapsed
Iter 247  :  Rows 46740-46930  :  2.227048873901367 seconds elapsed
Iter 248  :  Rows 46930-47120  :  2.214615821838379 seconds elapsed
Iter 249  :  Rows 47120-47310  :  2.2035562992095947 seconds elapsed
Iter 250  :  Rows 47310-47500  :  2.221041202545166 seconds elapsed
Iter 251  :  Rows 47500-47690  :  2.212592840194702 seconds elapsed
Iter 252  :  Rows 47690-47880  :  2.2212719917297363 seconds elapsed
Iter 253  :  Rows 47880-48070  :  2.2128329277038574 seconds elapsed
Iter 254  :  Rows 48070-48260  :  2.2214131355285645 seconds elapsed
Iter 255  :  Rows 48260-48450  :  2.2418320178985596 seconds elapsed
Iter 256  :  Rows 48450-48640  :  2.207494020462036 seconds elapsed
Iter 257  :  Rows 48640-48830  :  2.2243571281433105 seconds elapsed
Iter 258  :  Rows 48830-49020  :  2.2237

Iter 364  :  Rows 68970-69160  :  2.2157440185546875 seconds elapsed
Iter 365  :  Rows 69160-69350  :  2.206223964691162 seconds elapsed
Iter 366  :  Rows 69350-69540  :  2.2147817611694336 seconds elapsed
Iter 367  :  Rows 69540-69730  :  2.205334186553955 seconds elapsed
Iter 368  :  Rows 69730-69920  :  2.2416298389434814 seconds elapsed
Iter 369  :  Rows 69920-70110  :  2.208853006362915 seconds elapsed
Iter 370  :  Rows 70110-70300  :  2.234790086746216 seconds elapsed
Iter 371  :  Rows 70300-70490  :  2.2193338871002197 seconds elapsed
Iter 372  :  Rows 70490-70680  :  2.2389068603515625 seconds elapsed
Iter 373  :  Rows 70680-70870  :  2.2203521728515625 seconds elapsed
Iter 374  :  Rows 70870-71060  :  2.22337007522583 seconds elapsed
Iter 375  :  Rows 71060-71250  :  2.22879695892334 seconds elapsed
Iter 376  :  Rows 71250-71440  :  2.2092580795288086 seconds elapsed
Iter 377  :  Rows 71440-71630  :  2.2148258686065674 seconds elapsed
Iter 378  :  Rows 71630-71820  :  2.213398

Iter 484  :  Rows 91770-91960  :  2.199388027191162 seconds elapsed
Iter 485  :  Rows 91960-92150  :  2.234147071838379 seconds elapsed
Iter 486  :  Rows 92150-92340  :  2.2596731185913086 seconds elapsed
Iter 487  :  Rows 92340-92530  :  2.2020750045776367 seconds elapsed
Iter 488  :  Rows 92530-92720  :  2.240078926086426 seconds elapsed
Iter 489  :  Rows 92720-92910  :  2.2175557613372803 seconds elapsed
Iter 490  :  Rows 92910-93100  :  2.222337007522583 seconds elapsed
Iter 491  :  Rows 93100-93290  :  2.2415549755096436 seconds elapsed
Iter 492  :  Rows 93290-93480  :  2.2156970500946045 seconds elapsed
Iter 493  :  Rows 93480-93670  :  2.2221100330352783 seconds elapsed
Iter 494  :  Rows 93670-93860  :  2.22360897064209 seconds elapsed
Iter 495  :  Rows 93860-94050  :  2.2169930934906006 seconds elapsed
Iter 496  :  Rows 94050-94240  :  2.233288049697876 seconds elapsed
Iter 497  :  Rows 94240-94430  :  2.225677251815796 seconds elapsed
Iter 498  :  Rows 94430-94620  :  2.186295

...

Iter 3139  :  Rows 596220-596410  :  2.1941099166870117 seconds elapsed
Iter 3140  :  Rows 596410-596600  :  2.1683480739593506 seconds elapsed
Iter 3141  :  Rows 596600-596790  :  2.204949140548706 seconds elapsed
Iter 3142  :  Rows 596790-596980  :  2.2020559310913086 seconds elapsed
Iter 3143  :  Rows 596980-597170  :  2.2040371894836426 seconds elapsed
Iter 3144  :  Rows 597170-597360  :  2.2152328491210938 seconds elapsed
Iter 3145  :  Rows 597360-597550  :  2.2247607707977295 seconds elapsed
Iter 3146  :  Rows 597550-597740  :  2.1970818042755127 seconds elapsed
Iter 3147  :  Rows 597740-597930  :  2.2194628715515137 seconds elapsed
Iter 3148  :  Rows 597930-598120  :  2.197882890701294 seconds elapsed
Iter 3149  :  Rows 598120-598310  :  2.2116098403930664 seconds elapsed
Iter 3150  :  Rows 598310-598500  :  2.199924945831299 seconds elapsed
Iter 3151  :  Rows 598500-598690  :  2.207667112350464 seconds elapsed
Iter 3152  :  Rows 598690-598880  :  2.1882309913635254 seconds elap

Iter 3254  :  Rows 618070-618260  :  2.197087049484253 seconds elapsed
Iter 3255  :  Rows 618260-618450  :  2.1925032138824463 seconds elapsed
Iter 3256  :  Rows 618450-618640  :  2.2242579460144043 seconds elapsed
Iter 3257  :  Rows 618640-618830  :  2.20284104347229 seconds elapsed
Iter 3258  :  Rows 618830-619020  :  2.227349042892456 seconds elapsed
Iter 3259  :  Rows 619020-619210  :  2.2196412086486816 seconds elapsed
Iter 3260  :  Rows 619210-619400  :  2.2145609855651855 seconds elapsed
Iter 3261  :  Rows 619400-619590  :  2.210469961166382 seconds elapsed
Iter 3262  :  Rows 619590-619780  :  2.2171850204467773 seconds elapsed
Iter 3263  :  Rows 619780-619970  :  2.1994869709014893 seconds elapsed
Iter 3264  :  Rows 619970-620160  :  2.2175540924072266 seconds elapsed
Iter 3265  :  Rows 620160-620350  :  2.20202374458313 seconds elapsed
Iter 3266  :  Rows 620350-620540  :  2.2026920318603516 seconds elapsed
Iter 3267  :  Rows 620540-620730  :  2.2014338970184326 seconds elapsed

Iter 3369  :  Rows 639920-640110  :  2.2196309566497803 seconds elapsed
Iter 3370  :  Rows 640110-640300  :  2.2074596881866455 seconds elapsed
Iter 3371  :  Rows 640300-640490  :  2.194992780685425 seconds elapsed
Iter 3372  :  Rows 640490-640680  :  2.2052550315856934 seconds elapsed
Iter 3373  :  Rows 640680-640870  :  2.212114095687866 seconds elapsed
Iter 3374  :  Rows 640870-641060  :  2.2077279090881348 seconds elapsed
Iter 3375  :  Rows 641060-641250  :  2.1966350078582764 seconds elapsed
Iter 3376  :  Rows 641250-641440  :  2.22259521484375 seconds elapsed
Iter 3377  :  Rows 641440-641630  :  2.2153420448303223 seconds elapsed
Iter 3378  :  Rows 641630-641820  :  2.20792818069458 seconds elapsed
Iter 3379  :  Rows 641820-642010  :  2.191051959991455 seconds elapsed
Iter 3380  :  Rows 642010-642200  :  2.182781219482422 seconds elapsed
Iter 3381  :  Rows 642200-642390  :  2.2281270027160645 seconds elapsed
Iter 3382  :  Rows 642390-642580  :  2.1979172229766846 seconds elapsed


Iter 3484  :  Rows 661770-661960  :  2.227602958679199 seconds elapsed
Iter 3485  :  Rows 661960-662150  :  2.369614839553833 seconds elapsed
Iter 3486  :  Rows 662150-662340  :  2.2668330669403076 seconds elapsed
Iter 3487  :  Rows 662340-662530  :  2.200220823287964 seconds elapsed
Iter 3488  :  Rows 662530-662720  :  2.220158100128174 seconds elapsed
Iter 3489  :  Rows 662720-662910  :  2.2051148414611816 seconds elapsed
Iter 3490  :  Rows 662910-663100  :  2.1809821128845215 seconds elapsed
Iter 3491  :  Rows 663100-663290  :  2.18871808052063 seconds elapsed
Iter 3492  :  Rows 663290-663480  :  2.1904211044311523 seconds elapsed
Iter 3493  :  Rows 663480-663670  :  2.186706066131592 seconds elapsed
Iter 3494  :  Rows 663670-663860  :  2.1840600967407227 seconds elapsed
Iter 3495  :  Rows 663860-664050  :  2.2012507915496826 seconds elapsed
Iter 3496  :  Rows 664050-664240  :  2.216677188873291 seconds elapsed
Iter 3497  :  Rows 664240-664430  :  2.2183148860931396 seconds elapsed


Iter 3599  :  Rows 683620-683810  :  2.218035936355591 seconds elapsed
Iter 3600  :  Rows 683810-684000  :  2.214689254760742 seconds elapsed
Iter 3601  :  Rows 684000-684190  :  2.2127299308776855 seconds elapsed
Iter 3602  :  Rows 684190-684380  :  2.21683406829834 seconds elapsed
Iter 3603  :  Rows 684380-684570  :  2.2432961463928223 seconds elapsed
Iter 3604  :  Rows 684570-684760  :  2.216130256652832 seconds elapsed
Iter 3605  :  Rows 684760-684950  :  2.225377082824707 seconds elapsed
Iter 3606  :  Rows 684950-685140  :  2.2185637950897217 seconds elapsed
Iter 3607  :  Rows 685140-685330  :  2.226839065551758 seconds elapsed
Iter 3608  :  Rows 685330-685520  :  2.2242050170898438 seconds elapsed
Iter 3609  :  Rows 685520-685710  :  2.226043939590454 seconds elapsed
Iter 3610  :  Rows 685710-685900  :  2.2092480659484863 seconds elapsed
Iter 3611  :  Rows 685900-686090  :  2.228442907333374 seconds elapsed
Iter 3612  :  Rows 686090-686280  :  2.2139830589294434 seconds elapsed
I

In [6]:
del cc_array

In [7]:
cc_wed = list(chain.from_iterable(cc_wed_chunked))
print(len(cc_wed))

703532


In [8]:
print(f"""Max WED: {max(cc_wed)}
Min WED: {min(cc_wed)}
Mean: {np.mean(cc_wed)}
Std Dev: {np.std(cc_wed)}""")

Max WED: 0.8484386464904401
Min WED: 0.1815071659402093
Mean: 0.27598083037616433
Std Dev: 0.05075569724079787


In [9]:
wed_indexed = list(enumerate(cc_wed))
wed_indexed10k = sorted(wed_indexed, key=itemgetter(1))[-10000:]
wed_indices10k = [tup[0] for tup in wed_indexed10k]
wed_scores10k = [tup[1] for tup in wed_indexed10k]

In [10]:
print(f"""Max WED in subsample: {max(wed_scores10k)}
Min WED: {min(wed_scores10k)}
Mean: {np.mean(wed_scores10k)}
Std Dev: {np.std(wed_scores10k)}""")

Max WED in subsample: 0.8484386464904401
Min WED: 0.4253332667618559
Mean: 0.4456780084688563
Std Dev: 0.028223730462232817


In [11]:
with open(os.path.join("data","cc_news.json"), "r") as infile:
    cc_news = json.load(infile)

print(len(cc_news))

703532


In [12]:
%%time

cc_wed10k = [
    article for idx, article in enumerate(cc_news) if idx in wed_indices10k
]

CPU times: user 48.2 s, sys: 147 ms, total: 48.4 s
Wall time: 48.4 s


In [13]:
with open(os.path.join("data", "cc_wed10k.json"), "w") as outfile:
    json.dump(cc_wed10k, outfile)

In [14]:
with open(os.path.join("data", "cc_wed10k.txt"), "w") as outfile:
    for article in cc_wed10k:
        outfile.write(article)